In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pointCollection as pc


In [2]:
%matplotlib notebook


In [3]:
thefile='/home/ben/git_repos/surfaceChange/ATL15.h5'
thefile='/Volumes/ice2/ben/ATL14_test/001/ATL15.h5'
V0={}
dV0={}
A0={}
for group in ['', '_10km','_20km', '_40km']:   
    D15=pc.grid.data().from_h5(thefile, group='height_change'+group)
    field='cell_area'+group
    A=getattr(D15, field)
    dh=getattr(D15, 'delta_h'+group)
    dhdt=getattr(D15, 'dhdt_lag1'+group)
    A[A>1.e16]=np.NaN
    A0[group]=np.nansum(A)
    V0[group] = np.array([np.nansum(A*dh[:,:,ii]) for ii in range(dh.shape[2])])
    dV0[group] = np.array([np.nansum(A*dhdt[:,:,ii]) for ii in range(dhdt.shape[2])])


In [8]:
Acell = {'':1.e3**2, '_10km':1.e4**2, '_20km':2.e4**2, '_40km':4.e4**2}

{key:(A0[key]-A0[''])/Acell[key] for key in A0}

{'': 0.0, '_10km': 43.69416192, '_20km': 10.4005632, '_40km': 2.60022272}

In [5]:
dhdt.shape

(67, 37, 8)

In [6]:
{key:V0[key]-V0[''] for key in V0}

{'': array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 '_10km': array([-1.4266008e+10, -3.7677761e+10, -5.9709227e+10,  0.0000000e+00,
         6.9612503e+10,  4.1333555e+10,  1.9744096e+10,  5.1461751e+10,
         1.2479916e+11], dtype=float32),
 '_20km': array([-1.8710528e+10, -5.0664767e+10, -7.9692595e+10,  0.0000000e+00,
         9.2094857e+10,  5.3990851e+10,  2.5688769e+10,  6.7975348e+10,
         1.6715625e+11], dtype=float32),
 '_40km': array([-2.62514278e+10, -6.96787927e+10, -1.07284955e+11,  0.00000000e+00,
         1.19699669e+11,  7.04660275e+10,  3.35956541e+10,  8.94089626e+10,
         2.21713793e+11], dtype=float32)}

In [7]:
{key:dV0[key]-dV0[''] for key in V0}

{'': array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 '_10km': array([-9.3648191e+10, -8.8125899e+10,  2.3883691e+11,  2.7845001e+11,
        -1.1311540e+11, -8.6358065e+10,  1.2687088e+11,  2.9334962e+11],
       dtype=float32),
 '_20km': array([-1.27818465e+11, -1.16111049e+11,  3.18770381e+11,  3.68379429e+11,
        -1.52415371e+11, -1.13208672e+11,  1.69146647e+11,  3.96723356e+11],
       dtype=float32),
 '_40km': array([-1.7371090e+11, -1.5042429e+11,  4.2913982e+11,  4.7879868e+11,
        -1.9693411e+11, -1.4748169e+11,  2.2325330e+11,  5.2921939e+11],
       dtype=float32)}

In [ ]:
plt.figure(); 
plt.plot(dV0['']/np.nansum(A0['']))
plt.plot(dV0['_10km']/np.nansum(A0['_10km']))
plt.plot(dV0['_20km']/np.nansum(A0['_20km']))
plt.plot(dV0['_40km']/np.nansum(A0['_40km']))
plt.legend(['1km', '10km', '20km', '40km'])

In [ ]:
hfig=plt.figure();
hax=hfig.subplots(1, 4, sharex=True, sharey=True)
for ii, av in enumerate(A0.keys()):
    D15=pc.grid.data().from_h5(thefile, group='height_change'+av)
    temp=getattr(D15, 'cell_area'+av)
    temp[temp>1.e15]=np.NaN
    hax[ii].imshow(getattr(D15, 'cell_area'+av), extent=D15.extent, origin='lower')

In [ ]:
D15=pc.grid.data().from_h5(thefile, group='height_change'+group)
D15

In [ ]:
av

In [ ]:
! h5ls /home/ben/git_repos/surfaceChange/ATL15.h5/height_change_10km